# Examples for data structures used for event lists

## Binary Heap

In [13]:
import random

class BinaryHeap:
    def __init__(self):
        self.heap = []

    def parent(self, index):
        return (index - 1) // 2

    def left_child(self, index):
        return 2 * index + 1

    def right_child(self, index):
        return 2 * index + 2

    def insert(self, key):
        self.heap.append(key)
        self.heapify_up(len(self.heap) - 1)

    def heapify_up(self, index):
        while index != 0 and self.heap[self.parent(index)] > self.heap[index]:
            self.heap[self.parent(index)], self.heap[index] = self.heap[index], self.heap[self.parent(index)]
            index = self.parent(index)

    def extract_min(self):
        if len(self.heap) == 0:
            return None
        root = self.heap[0]
        self.heap[0] = self.heap[-1]
        self.heap.pop()
        self.heapify_down(0)
        return root

    def heapify_down(self, index):
        smallest = index
        left = self.left_child(index)
        right = self.right_child(index)

        if left < len(self.heap) and self.heap[left] < self.heap[smallest]:
            smallest = left

        if right < len(self.heap) and self.heap[right] < self.heap[smallest]:
            smallest = right

        if smallest != index:
            self.heap[smallest], self.heap[index] = self.heap[index], self.heap[smallest]
            self.heapify_down(smallest)

    def visualize(self, index=0, indent=""):
        if index < len(self.heap):
            right_index = self.right_child(index)
            if right_index < len(self.heap):
                self.visualize(right_index, indent + "    ")

            print(indent + str(self.heap[index]))

            left_index = self.left_child(index)
            if left_index < len(self.heap):
                self.visualize(left_index, indent + "    ")


# Example usage
if __name__ == "__main__":
    heap = BinaryHeap()
    for x in range(20):
        heap.insert(4 + int(random.random() * 100))    
    
    heap.visualize()  # Visualizes the binary heap structure
    print("Insert 4")  # Output: 4
    heap.insert(4)
    heap.visualize()  # Visualizes the binary heap structure
    print(f"Minimum: {heap.extract_min()}")  # Output: 4
    heap.visualize()  # Visualizes the binary heap structure after extraction


            71
        66
            83
    20
            65
        27
            41
11
            62
        19
            55
                82
    17
                71
            55
                91
        27
                98
            82
                102
Insert 4
            71
        66
            83
    20
            65
        27
            41
4
            62
        17
                55
            19
                82
    11
                71
            55
                91
        27
                98
            82
                102
Minimum: 4
            71
        66
            83
    20
            65
        27
            41
11
            62
        19
            55
                82
    17
                71
            55
                91
        27
                98
            82
                102


## AVL Tree

In [14]:
import matplotlib.pyplot as plt
import networkx as nx
import ipywidgets as widgets
from IPython.display import display

class AVLNode:
    def __init__(self, key):
        self.key = key
        self.height = 1
        self.left = None
        self.right = None

class AVLTree:
    def get_height(self, node):
        if not node:
            return 0
        return node.height

    def get_balance(self, node):
        if not node:
            return 0
        return self.get_height(node.left) - self.get_height(node.right)

    def left_rotate(self, z):
        y = z.right
        T2 = y.left

        y.left = z
        z.right = T2

        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def right_rotate(self, z):
        y = z.left
        T3 = y.right

        y.right = z
        z.left = T3

        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def insert(self, root, key):
        if not root:
            return AVLNode(key)
        elif key < root.key:
            root.left = self.insert(root.left, key)
        else:
            root.right = self.insert(root.right, key)

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        balance = self.get_balance(root)

        if balance > 1 and key < root.left.key:
            return self.right_rotate(root)

        if balance < -1 and key > root.right.key:
            return self.left_rotate(root)

        if balance > 1 and key > root.left.key:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        if balance < -1 and key < root.right.key:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    def pre_order(self, root):
        res = []
        if root:
            res.append(root.key)
            res = res + self.pre_order(root.left)
            res = res + self.pre_order(root.right)
        return res

def visualize_tree(root, ax):
    def add_edges(graph, node, pos=None, x=0, y=0, layer=1):
        if pos is None:
            pos = {}
        pos[node.key] = (x, y)
        if node.left:
            graph.add_edge(node.key, node.left.key)
            l = x - 1 / layer
            pos = add_edges(graph, node.left, x=l, y=y-1, pos=pos, layer=layer+1)
        if node.right:
            graph.add_edge(node.key, node.right.key)
            r = x + 1 / layer
            pos = add_edges(graph, node.right, x=r, y=y-1, pos=pos, layer=layer+1)
        return pos
    
    graph = nx.DiGraph()
    pos = add_edges(graph, root)
    
    nx.draw(graph, pos, ax=ax, with_labels=True, node_size=500, node_color='lightblue', font_size=10, font_weight='bold')
    ax.set_title('AVL Tree')

def update_visualization(root, ax):    
    visualize_tree(root, ax)
    
avl_tree = AVLTree()
root = None

input_number = widgets.IntText(description="Add number:")
add_button = widgets.Button(description="Add to AVL Tree")

output = widgets.Output()

def on_add_button_clicked(b):
    global root
        
    with output:
        output.clear_output()
        fig, ax = plt.subplots(1, 2, figsize=(8, 6))
        number = input_number.value        
        if root != None:
            update_visualization(root, ax[0])
        root = avl_tree.insert(root, number)
        update_visualization(root, ax[1])
        plt.show()

add_button.on_click(on_add_button_clicked)

display(input_number, add_button, output)


IntText(value=0, description='Add number:')

Button(description='Add to AVL Tree', style=ButtonStyle())

Output()

## Skip List

In [16]:
import random

class SkipNode:
    def __init__(self, key, level):
        self.key = key
        self.forward = [None] * (level + 1)

class SkipList:
    def __init__(self, max_level):
        self.max_level = max_level
        self.head = SkipNode(-1, self.max_level)
        self.level = 0

    def random_level(self):
        level = 0
        while random.random() < 0.5 and level < self.max_level:
            level += 1
        return level

    def insert(self, key):
        update = [None] * (self.max_level + 1)
        current = self.head

        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

        level = self.random_level()
        if level > self.level:
            for i in range(self.level + 1, level + 1):
                update[i] = self.head
            self.level = level

        new_node = SkipNode(key, level)
        for i in range(level + 1):
            new_node.forward[i] = update[i].forward[i]
            update[i].forward[i] = new_node

    def search(self, key):
        current = self.head
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
        current = current.forward[0]
        if current and current.key == key:
            return True
        return False

    def visualize(self):
        print("Skip List Levels:")
        for level in range(self.level + 1):
            current = self.head.forward[level]
            print("Level {}: ".format(level), end="")
            while current:
                print(current.key, end=" ")
                current = current.forward[level]
            print()

# Example usage
if __name__ == "__main__":
    skiplist = SkipList(3)
    for x in range(50):
        skiplist.insert(int(random.random() * 1000))
    
    skiplist.visualize()

    print(skiplist.search(6))  
    print(skiplist.search(4))  


Skip List Levels:
Level 0: 43 59 66 73 79 101 105 146 168 178 183 249 272 294 303 311 327 342 349 365 379 383 404 438 451 467 485 497 511 519 528 529 534 588 592 619 684 754 762 773 795 800 858 869 953 965 968 978 996 998 
Level 1: 43 59 66 73 79 105 168 178 183 249 294 303 327 383 451 467 485 497 511 519 528 529 754 800 869 953 965 978 996 998 
Level 2: 59 73 79 105 168 183 327 511 519 529 800 869 953 978 996 998 
Level 3: 327 529 800 953 998 
False
False
